# Building A Chatbot Using GROQ and Langchain

##### This project demonstrates the creation of a chatbot utilizing Groq technology and LangChain for conversational systems. The primary goals are to effectively manage dialogues and develop a responsive AI assistant. This chatbot is built on the Groq API using the Gemma2-9b-It model. 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000187F6CD6230>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000187F6CD6EC0>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Chukwuemeka and I am a Chief AI Engineer")])

AIMessage(content="Hello Chukwuemeka, it's a pleasure to meet you! \n\nBeing a Chief AI Engineer is a fascinating role.  What are some of the most exciting projects you're working on?  \n\nI'm eager to learn more about your work in the field of AI.\n", response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 26, 'total_tokens': 90, 'completion_time': 0.116363636, 'prompt_time': 0.00015086, 'queue_time': 0.01365787, 'total_time': 0.116514496}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1188e009-56e1-430b-aa3e-e3370d33244e-0', usage_metadata={'input_tokens': 26, 'output_tokens': 64, 'total_tokens': 90})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Chukwuemeka and I am a Chief AI Engineer"),
        AIMessage(content="Hello Chukwuemeka! It's pleasure to meet you. \n\nAs a Chief AI Engineer is a facnating role, What are some of the most exciting projects you're working on? \n\nI'm eager to learn more about your work in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Chukwuemeka, and you are a Chief AI Engineer!  \n\nIs there anything else you'd like to know or discuss? 😊 \n", response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 110, 'total_tokens': 146, 'completion_time': 0.065454545, 'prompt_time': 0.003437148, 'queue_time': 0.010190232, 'total_time': 0.068891693}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-4b9b1eca-11a2-4156-b101-e316ec5a6015-0', usage_metadata={'input_tokens': 110, 'output_tokens': 36, 'total_tokens': 146})

### Message History
let's use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [6]:
config={"configurable":{"session_id":"chat1"}}

In [7]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Chukwuemeka and I am a Chief AI Engineer")],
    config=config
)

In [8]:
response.content

"It's great to meet you, Chukwuemeka! That's a fantastic title – Chief AI Engineer.  \n\nWhat kind of exciting projects are you working on? \n\nI'm always eager to learn more about the innovative things people are doing with AI.  \n\n"

In [9]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Chukwuemeka.  \n\nI remember it from our introduction! 😊 \n\n\n', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 101, 'total_tokens': 124, 'completion_time': 0.041818182, 'prompt_time': 0.005237841, 'queue_time': 0.008377519, 'total_time': 0.047056023}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-eb01f50f-f3e0-49b8-b953-2d44f3a1dd6e-0', usage_metadata={'input_tokens': 101, 'output_tokens': 23, 'total_tokens': 124})

In [10]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to use it! 😊  \n\n"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is James")],
    config=config1
)
response.content

'Hi James! \n\nNice to meet you. What can I do for you today? 😊 \n\n\n\n'

In [14]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'You said your name is James! 😊  \n\nIs there anything else I can help you with? \n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [15]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [20]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is James")]})

AIMessage(content="Hello James, it's nice to meet you!  \n\nIs there anything I can help you with today? 😄 \n", response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 31, 'total_tokens': 59, 'completion_time': 0.050909091, 'prompt_time': 0.000345888, 'queue_time': 0.014272041999999999, 'total_time': 0.051254979}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d4ee4e51-c967-4e79-95bf-b53e3bf1181c-0', usage_metadata={'input_tokens': 31, 'output_tokens': 28, 'total_tokens': 59})

In [21]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [22]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Chukwuemeka")],
    config=config
)

response

AIMessage(content="Hello Chukwuemeka!  \n\nIt's nice to meet you.  \n\nI'm ready to answer your questions to the best of my ability. What can I help you with? 😊 \n\n", response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 35, 'total_tokens': 82, 'completion_time': 0.085454545, 'prompt_time': 0.000360089, 'queue_time': 0.01299225, 'total_time': 0.085814634}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7949ba14-1a09-473a-b8f1-a1ed7f386e86-0', usage_metadata={'input_tokens': 35, 'output_tokens': 47, 'total_tokens': 82})

In [23]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Chukwuemeka!  \n\nI remember that from when you first introduced yourself.  😄  \n\n\n\n'

In [24]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [26]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is James")],"language":"french"})
response.content

"Enchanté, James ! Comment puis-je vous aider aujourd'hui ? \n"

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [27]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [28]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am James")],"language":"french"},
    config=config
)
repsonse.content

"Bonjour James, enchanté de te rencontrer ! 👋  Comment puis-je vous aider aujourd'hui ? 😊 \n"

In [29]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "french"},
    config=config,
)

In [30]:
response.content

'Votre nom est James.  \n\n😊  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [31]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm Tim"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [32]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a helpful assistant, I don't have personal preferences like taste buds! \n\nTo figure out what ice cream *you* like, think about:\n\n* **Your favorite flavors:** Chocolate, vanilla, strawberry?  Something more unique like mint chip or cookie dough?\n* **What textures you enjoy:** Creamy, chunky, smooth, or icy?\n* **Toppings:** Do you like sprinkles, nuts, sauces, or fruit? \n\nLet me know if you want to brainstorm some ice cream ideas!  🍦😊 \n\n\n\n"

In [33]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked:  "whats 2 + 2" \n\n\nLet me know if you have any other math problems you\'d like to solve! 😄  \n'

In [34]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [36]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name. 😊\n\nIf you'd like to tell me your name, I'd be happy to know!\n"

In [37]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'As a large language model, I have no memory of past conversations.  To help me answer your math problem, please tell me what it is! 😊  \n\n'